In [ ]:
from keras.preprocessing import image
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input

import numpy as np
from os.path import join
import os

In [ ]:
regions = ['borde_rural', 'borde_soacha', 'castries', 'dennery', 'gros_islet', 'mixco_3', 'mixco_1_and_ebenezer']

In [ ]:
region = 'mixco_3'
roof_dir = join('..', '..', 'data', region, 'roofs_train')
materials = {'healthy_metal':0, 'irregular_metal':1, 'concrete_cement':2, 'incomplete':3, 'other':4}

### Calculate features

This uses a pretrained network to compute features from the images, which are later classified.
First, load the model, from which we remove the top layer

In [ ]:
model = ResNet50(weights='imagenet', include_top=False, pooling='max')
model.summary()

Traverse all images in their corresponding folders which indicate the material and use `model.predict()` to get the features.

In [ ]:
# Count the number of roofs
nof_roofs = 0;
for material in materials.keys():
    material_fp = join(roof_dir, material)
    nof_this = len([name for name in os.listdir(material_fp) if os.path.isfile(join(material_fp, name))])
    nof_roofs = nof_roofs + nof_this

labels = []
resnet50_feature_matrix = np.zeros((nof_roofs, 2048), dtype=float)

# Walk through all images
i = 0;
for material in materials.keys():
    material_fp = join(roof_dir, material)
    for root, dirs, files in os.walk(material_fp):
        for file in files:
            img_fp = join(material_fp, file)
            label = materials[material]
            labels.append(label)
            print(img_fp, "labeled as ", material, ":", label)
            
            # Pad if size is too small, preprocess
            img = image.load_img(img_fp, target_size=(224, 224))
            img_data = image.img_to_array(img)
            img_data = np.expand_dims(img_data, axis=0)
            img_data = preprocess_input(img_data)
            
            # Compute features 
            resnet50_feature = model.predict(img_data)
            resnet50_feature_np = np.array(resnet50_feature)
            resnet50_feature_matrix[i] = resnet50_feature_np.flatten()
            i = i + 1

In [ ]:
unique, index, count = np.unique(resnet50_feature_matrix,
                                 axis = 0,
                                 return_index=True,
                                 return_counts=True)

In [ ]:
index.sort()
print(index)

In [ ]:
import pickle
from os import makedirs
from os.path import exists

pickle_path = join('..', '..', 'pickles')
if not exists(pickle_path):
    makedirs(pickle_path)

with open(join(pickle_path, 'resnet50_feature_matrix_' + region + '.pkl' ), 'wb') as f:
    pickle.dump(resnet50_feature_matrix, f)
with open(join(pickle_path, 'labels_' + region + '.pkl' ), 'wb') as f:
    pickle.dump(np.asarray(labels), f)

### Visualize the features

Reduce the dimensionality to 2D using tSNE (https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding)

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib
from sklearn.decomposition import PCA
%matplotlib inline

In [ ]:
def plot_tSNE(features, labels=None, number_of_materials=5):
    if labels is None:
        labels = np.zeros((features.shape[0]))

    # Visualization_
    #pca_object = PCA(n_components=50)
    #pca_features = pca_object.fit_transform(features)
    tsne_features = TSNE(n_components=2).fit_transform(features)

    # define the colormap
    cmap = plt.cm.jet
    # extract all colors from the .jet map
    cmaplist = [cmap(i) for i in range(cmap.N)]
    # create the new map
    cmap = cmap.from_list('Custom cmap', cmaplist, cmap.N)

    # define the bins and normalize
    bounds = np.linspace(0, number_of_materials, number_of_materials + 1)
    norm = matplotlib.colors.BoundaryNorm(bounds, cmap.N)
    
    print(tsne_features.shape)
    
    # Plot D-Vectors
    plt.figure()
    scat = plt.scatter(tsne_features[:, 0], tsne_features[:, 1], c=labels, cmap=cmap, norm=norm)
    cb = plt.colorbar(scat, spacing='proportional', ticks=bounds)

In [ ]:
plot_tSNE(resnet50_feature_matrix, labels)

In module:

In [ ]:
import extracting_features

region = 'borde_soacha'
resnet50_feature_matrix, labels = extracting_features.extract_features(region)

Using TensorFlow backend.


..\..\data\borde_soacha\roofs_train\healthy_metal\7a1c5468.tif labeled as  healthy_metal : 0
..\..\data\borde_soacha\roofs_train\healthy_metal\7a1c6340.tif labeled as  healthy_metal : 0
..\..\data\borde_soacha\roofs_train\healthy_metal\7a1c65c0.tif labeled as  healthy_metal : 0
..\..\data\borde_soacha\roofs_train\healthy_metal\7a1c75a6.tif labeled as  healthy_metal : 0
..\..\data\borde_soacha\roofs_train\healthy_metal\7a1c7a2e.tif labeled as  healthy_metal : 0
..\..\data\borde_soacha\roofs_train\healthy_metal\7a1c7c04.tif labeled as  healthy_metal : 0
..\..\data\borde_soacha\roofs_train\healthy_metal\7a1c7e70.tif labeled as  healthy_metal : 0
..\..\data\borde_soacha\roofs_train\healthy_metal\7a1c81ea.tif labeled as  healthy_metal : 0
..\..\data\borde_soacha\roofs_train\healthy_metal\7a1c8262.tif labeled as  healthy_metal : 0
..\..\data\borde_soacha\roofs_train\healthy_metal\7a1c8b04.tif labeled as  healthy_metal : 0
..\..\data\borde_soacha\roofs_train\healthy_metal\7a1c932e.tif labeled

..\..\data\borde_soacha\roofs_train\healthy_metal\7a1d9602.tif labeled as  healthy_metal : 0
..\..\data\borde_soacha\roofs_train\healthy_metal\7a1d9792.tif labeled as  healthy_metal : 0
..\..\data\borde_soacha\roofs_train\healthy_metal\7a1d97f6.tif labeled as  healthy_metal : 0
..\..\data\borde_soacha\roofs_train\healthy_metal\7a1d98c8.tif labeled as  healthy_metal : 0
..\..\data\borde_soacha\roofs_train\healthy_metal\7a1d9990.tif labeled as  healthy_metal : 0
..\..\data\borde_soacha\roofs_train\healthy_metal\7a1d9ac6.tif labeled as  healthy_metal : 0
..\..\data\borde_soacha\roofs_train\healthy_metal\7a1d9c60.tif labeled as  healthy_metal : 0
..\..\data\borde_soacha\roofs_train\healthy_metal\7a1d9cc4.tif labeled as  healthy_metal : 0
..\..\data\borde_soacha\roofs_train\healthy_metal\7a1da0c0.tif labeled as  healthy_metal : 0
..\..\data\borde_soacha\roofs_train\healthy_metal\7a1da192.tif labeled as  healthy_metal : 0
..\..\data\borde_soacha\roofs_train\healthy_metal\7a1daa48.tif labeled

..\..\data\borde_soacha\roofs_train\healthy_metal\7a1e98f4.tif labeled as  healthy_metal : 0
..\..\data\borde_soacha\roofs_train\healthy_metal\7a1e9f16.tif labeled as  healthy_metal : 0
..\..\data\borde_soacha\roofs_train\healthy_metal\7a1ea18c.tif labeled as  healthy_metal : 0
..\..\data\borde_soacha\roofs_train\healthy_metal\7a1ea1fa.tif labeled as  healthy_metal : 0
..\..\data\borde_soacha\roofs_train\healthy_metal\7a1eaa92.tif labeled as  healthy_metal : 0
..\..\data\borde_soacha\roofs_train\healthy_metal\7a1eab64.tif labeled as  healthy_metal : 0
..\..\data\borde_soacha\roofs_train\healthy_metal\7a1eabc8.tif labeled as  healthy_metal : 0
..\..\data\borde_soacha\roofs_train\healthy_metal\7a1eae3e.tif labeled as  healthy_metal : 0
..\..\data\borde_soacha\roofs_train\healthy_metal\7a1eb4ce.tif labeled as  healthy_metal : 0
..\..\data\borde_soacha\roofs_train\healthy_metal\7a1eb596.tif labeled as  healthy_metal : 0
..\..\data\borde_soacha\roofs_train\healthy_metal\7a1ebc1c.tif labeled

..\..\data\borde_soacha\roofs_train\healthy_metal\7a1fee34.tif labeled as  healthy_metal : 0
..\..\data\borde_soacha\roofs_train\healthy_metal\7a1feea2.tif labeled as  healthy_metal : 0
..\..\data\borde_soacha\roofs_train\healthy_metal\7a1fef06.tif labeled as  healthy_metal : 0
..\..\data\borde_soacha\roofs_train\healthy_metal\7a1fef74.tif labeled as  healthy_metal : 0
..\..\data\borde_soacha\roofs_train\healthy_metal\7a1ff0aa.tif labeled as  healthy_metal : 0
..\..\data\borde_soacha\roofs_train\healthy_metal\7a1ff1ea.tif labeled as  healthy_metal : 0
..\..\data\borde_soacha\roofs_train\healthy_metal\7a1ff258.tif labeled as  healthy_metal : 0
..\..\data\borde_soacha\roofs_train\healthy_metal\7a1ff38e.tif labeled as  healthy_metal : 0
..\..\data\borde_soacha\roofs_train\healthy_metal\7a1ff6cc.tif labeled as  healthy_metal : 0
..\..\data\borde_soacha\roofs_train\healthy_metal\7a1ffc44.tif labeled as  healthy_metal : 0
..\..\data\borde_soacha\roofs_train\healthy_metal\7a1ffca8.tif labeled

...

..\..\data\borde_soacha\roofs_train\irregular_metal\7a3c4c78.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_train\irregular_metal\7a3c552e.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_train\irregular_metal\7a3c588a.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_train\irregular_metal\7a3c5b78.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_train\irregular_metal\7a3c5c4a.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_train\irregular_metal\7a3c5f38.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_train\irregular_metal\7a3c600a.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_train\irregular_metal\7a3c6294.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_train\irregular_metal\7a3c6582.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_train\irregular_metal\7a3c69ba.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_

..\..\data\borde_soacha\roofs_train\irregular_metal\7a3dcb98.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_train\irregular_metal\7a3dd2fa.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_train\irregular_metal\7a3dd35e.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_train\irregular_metal\7a3ddd40.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_train\irregular_metal\7a3de01a.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_train\irregular_metal\7a3de2ea.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_train\irregular_metal\7a3de42a.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_train\irregular_metal\7a3de560.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_train\irregular_metal\7a3de5c4.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_train\irregular_metal\7a3df06e.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_

..\..\data\borde_soacha\roofs_train\irregular_metal\7a3f1caa.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_train\irregular_metal\7a3f2132.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_train\irregular_metal\7a3f2204.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_train\irregular_metal\7a3f2344.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_train\irregular_metal\7a3f2b78.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_train\irregular_metal\7a3f2be6.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_train\irregular_metal\7a3f2f92.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_train\irregular_metal\7a3f3000.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_train\irregular_metal\7a3f3078.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_train\irregular_metal\7a3f3398.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_

..\..\data\borde_soacha\roofs_train\irregular_metal\7a4085a4.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_train\irregular_metal\7a4090ee.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_train\irregular_metal\7a409152.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_train\irregular_metal\7a409efe.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_train\irregular_metal\7a40a408.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_train\irregular_metal\7a40a53e.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_train\irregular_metal\7a40ab4c.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_train\irregular_metal\7a40abba.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_train\irregular_metal\7a40b056.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_train\irregular_metal\7a40b2e0.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_

..\..\data\borde_soacha\roofs_train\irregular_metal\7a421040.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_train\irregular_metal\7a422788.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_train\irregular_metal\7a422936.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_train\irregular_metal\7a422e40.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_train\irregular_metal\7a4234e4.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_train\irregular_metal\7a423e26.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_train\irregular_metal\7a423fd4.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_train\irregular_metal\7a424b46.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_train\irregular_metal\7a4258ca.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_train\irregular_metal\7a425ae6.tif labeled as  irregular_metal : 1
..\..\data\borde_soacha\roofs_

In [ ]:
import pickle
from os import makedirs
from os.path import exists

pickle_path = join('..', '..', 'pickles')
if not exists(pickle_path):
    makedirs(pickle_path)

with open(join(pickle_path, 'resnet50_feature_matrix_' + region + '.pkl' ), 'wb') as f:
    pickle.dump(resnet50_feature_matrix, f)
with open(join(pickle_path, 'labels_' + region + '.pkl' ), 'wb') as f:
    pickle.dump(np.asarray(labels), f)